In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Bidirectional

In [2]:
# Sample dataset

data = pd.read_csv('./NLU_Dataset.csv')

input_text = input('Enter text : ')

In [3]:
# Tokenization and sequence padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text for intent, examples in data.items() for text in examples])
vocab_size = len(tokenizer.word_index) + 1

max_len = max(len(text.split()) for , examples in data.items() for text in examples)
sequences = tokenizer.texts_to_sequences([text for intent, examples in data.items() for text in examples])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post')

# One-hot encode labels
labels = np.array([i for i, intent in enumerate(data.keys()) for _ in data[intent]])

In [4]:
# LSTM model
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_len))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(len(data), activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training
model.fit(padded_sequences, labels, epochs=50, verbose=0)

D:\My\CS\Hackothons\Intellihack\Intellihack_CodeCrafters_1\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [5]:
# Testing
test_input = "Hi there, can you tell me the time?"
test_sequence = tokenizer.texts_to_sequences([test_input])
padded_test_sequence = pad_sequences(test_sequence, maxlen=max_len, padding='post')

predicted_probabilities = model.predict(padded_test_sequence)[0]
predicted_intent_index = np.argmax(predicted_probabilities)
confidence_score = predicted_probabilities[predicted_intent_index]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step


In [6]:
# Fallback mechanism
threshold = 0.7
if confidence_score >= threshold:
    intents = list(data.keys())
    predicted_intent = intents[predicted_intent_index]
    print(f"Intent: {predicted_intent}, Confidence: {confidence_score}")
else:
    print("Fallback: Intent could not be confidently determined.")

Intent: Example, Confidence: 0.9960881471633911
